In [1]:
import pandas as pd

# Cargo el archivo CSV
df_consum_2023 = pd.read_csv('2023_consum_bcn.csv', header=0, delimiter= ",")

df_consum_2023.head(10)

,Any,Data,Codi_Postal,Sector_Economic,Tram_Horari,Valor
0,2023,2023-01-01,8001,Indústria,De 00:00:00 a 05:59:59 h,644
1,2023,2023-01-01,8001,Indústria,De 06:00:00 a 11:59:59 h,702
2,2023,2023-01-01,8001,Indústria,De 12:00:00 a 17:59:59 h,783
3,2023,2023-01-01,8001,Indústria,De 18:00:00 a 23:59:59 h,768
4,2023,2023-01-01,8001,Indústria,No consta,57
5,2023,2023-01-01,8001,Residencial,De 00:00:00 a 05:59:59 h,21672
6,2023,2023-01-01,8001,Residencial,De 06:00:00 a 11:59:59 h,22619
7,2023,2023-01-01,8001,Residencial,De 12:00:00 a 17:59:59 h,34533
8,2023,2023-01-01,8001,Residencial,De 18:00:00 a 23:59:59 h,36591
9,2023,2023-01-01,8001,Residencial,No consta,4932


La composición del dataframe consta de 241125 filas y 6 columnas

In [2]:
df_consum_2023.shape

(241125, 6)

Reviso el tipo de datos que tienen en este momento para determinar si hay que realizar algunas modificaciones

In [3]:
df_consum_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241125 entries, 0 to 241124
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Any              241125 non-null  int64 
 1   Data             241125 non-null  object
 2   Codi_Postal      241125 non-null  int64 
 3   Sector_Economic  241125 non-null  object
 4   Tram_Horari      241125 non-null  object
 5   Valor            241125 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 11.0+ MB


Confirmo la presencia de valores nulos (no se encuentran)

In [4]:
df_consum_2023.isnull().sum()

Any                0
Data               0
Codi_Postal        0
Sector_Economic    0
Tram_Horari        0
Valor              0
dtype: int64

Cambio el formato de `Data` a `datetime`

In [5]:
df_consum_2023['Data']= pd.to_datetime(df_consum_2023['Data'], format='%Y-%m-%d')

Elimino la columna `Any`, la información relacionada con el año se encuentra dentro de la columna `fecha`.

In [6]:
df_consum_2023= df_consum_2023.drop('Any', axis=1)
df_consum_2023.head()

,Data,Codi_Postal,Sector_Economic,Tram_Horari,Valor
0,2023-01-01,8001,Indústria,De 00:00:00 a 05:59:59 h,644
1,2023-01-01,8001,Indústria,De 06:00:00 a 11:59:59 h,702
2,2023-01-01,8001,Indústria,De 12:00:00 a 17:59:59 h,783
3,2023-01-01,8001,Indústria,De 18:00:00 a 23:59:59 h,768
4,2023-01-01,8001,Indústria,No consta,57


La columna de `Valor`la cambio de formato a `float`

In [7]:
df_consum_2023["Valor"] = df_consum_2023["Valor"].astype(float)

Observo que existe una fila en los registros con el valor "No consta" dentro de los tramos horarios, aprovecharé las filas de ests registros para conste el consumo total del sector correspondiente durante el día. Aplicaré la suma de los 4 registros anteriores en la columna porque cada barrio/sector contiene 4 tramos horarios durante el día.

In [8]:
# Identifico las filas que incluyen el "No consta"
mask = df_consum_2023["Tram_Horari"] == "No consta"

# Aplico la suma de los 4 registros anteriores en la columna Valor
df_consum_2023.loc[mask, "Valor"] = df_consum_2023["Valor"].shift(1).rolling(window=4, min_periods=1).sum()

# Reemplazo "No consta" por "De 00:00:00 a 23:59:59 h" en Tram_Horari para que conste el consumo de todo el día
df_consum_2023.loc[mask, "Tram_Horari"] = "De 00:00:00 a 23:59:59 h"

# Compruebo la aplicación del tramo modificado
df_consum_2023.head(10)

,Data,Codi_Postal,Sector_Economic,Tram_Horari,Valor
0,2023-01-01,8001,Indústria,De 00:00:00 a 05:59:59 h,644.0
1,2023-01-01,8001,Indústria,De 06:00:00 a 11:59:59 h,702.0
2,2023-01-01,8001,Indústria,De 12:00:00 a 17:59:59 h,783.0
3,2023-01-01,8001,Indústria,De 18:00:00 a 23:59:59 h,768.0
4,2023-01-01,8001,Indústria,De 00:00:00 a 23:59:59 h,2897.0
5,2023-01-01,8001,Residencial,De 00:00:00 a 05:59:59 h,21672.0
6,2023-01-01,8001,Residencial,De 06:00:00 a 11:59:59 h,22619.0
7,2023-01-01,8001,Residencial,De 12:00:00 a 17:59:59 h,34533.0
8,2023-01-01,8001,Residencial,De 18:00:00 a 23:59:59 h,36591.0
9,2023-01-01,8001,Residencial,De 00:00:00 a 23:59:59 h,115415.0


Para evitar problemas al momento de importar los datos, en la columna `Sector_Economic` aparece la palabra `Indústria` con acento, lo elimino para evitar problemas con la codificación, en las visualizaciones esto puede cambiarse.

Cambio además las palabras `Serveis` por `servicios` y `No especificat` por `No especificado`

In [9]:
df_consum_2023['Sector_Economic'] = df_consum_2023['Sector_Economic'].replace('Indústria', 'Industria')
df_consum_2023['Sector_Economic'] = df_consum_2023['Sector_Economic'].replace('Serveis', 'Servicios')
df_consum_2023['Sector_Economic'] = df_consum_2023['Sector_Economic'].replace('No especificat', 'No especificado')

df_consum_2023.head()

,Data,Codi_Postal,Sector_Economic,Tram_Horari,Valor
0,2023-01-01,8001,Industria,De 00:00:00 a 05:59:59 h,644.0
1,2023-01-01,8001,Industria,De 06:00:00 a 11:59:59 h,702.0
2,2023-01-01,8001,Industria,De 12:00:00 a 17:59:59 h,783.0
3,2023-01-01,8001,Industria,De 18:00:00 a 23:59:59 h,768.0
4,2023-01-01,8001,Industria,De 00:00:00 a 23:59:59 h,2897.0


Al contener datos de distintas fuentes oficiales opto por unificar los nombres de las columnas en castellano.

In [10]:
df_consum_2023= df_consum_2023.set_axis(['fecha', 'codigo_postal', 'sector_economico', 'tramo_horario', 'consumo'], axis='columns')
df_consum_2023.head()

,fecha,codigo_postal,sector_economico,tramo_horario,consumo
0,2023-01-01,8001,Industria,De 00:00:00 a 05:59:59 h,644.0
1,2023-01-01,8001,Industria,De 06:00:00 a 11:59:59 h,702.0
2,2023-01-01,8001,Industria,De 12:00:00 a 17:59:59 h,783.0
3,2023-01-01,8001,Industria,De 18:00:00 a 23:59:59 h,768.0
4,2023-01-01,8001,Industria,De 00:00:00 a 23:59:59 h,2897.0


Para poder trabajar fácilmente los tramos horarios modifico la columna tramo horario codificando cada uno de ellos con el tramo horario correspondiente que pertenecerá a una tabla que contendrá la información detallada y así reducir la dimensión del archivo exportado al final. 

In [11]:
# Defino el diccionario de mapeo
mapeo_tramo = {
    "De 00:00:00 a 05:59:59 h": "1",
    "De 06:00:00 a 11:59:59 h": "2",
    "De 12:00:00 a 17:59:59 h": "3",
    "De 18:00:00 a 23:59:59 h": "4",
    "De 00:00:00 a 23:59:59 h": "5"
}

# Creo la nueva columna "tramo_dia" usando el mapeo
df_consum_2023['tramo_horario'] = df_consum_2023['tramo_horario'].map(mapeo_tramo)

df_consum_2023.head()

,fecha,codigo_postal,sector_economico,tramo_horario,consumo
0,2023-01-01,8001,Industria,1,644.0
1,2023-01-01,8001,Industria,2,702.0
2,2023-01-01,8001,Industria,3,783.0
3,2023-01-01,8001,Industria,4,768.0
4,2023-01-01,8001,Industria,5,2897.0


Observo que los códigos postales no incluyen el cero inicial, los códigos postales constan de 5 dígitos y considero importante mantener ese formato que suele ser el habitual para identificarlos, agrego el cero dentro de los registros de esa columna

In [12]:
# Agrego un cero al inicio de los valores de la columna "Codi_Postal"
df_consum_2023['codigo_postal'] = '0' + df_consum_2023['codigo_postal'].astype(str)

df_consum_2023.head()

,fecha,codigo_postal,sector_economico,tramo_horario,consumo
0,2023-01-01,08001,Industria,1,644.0
1,2023-01-01,08001,Industria,2,702.0
2,2023-01-01,08001,Industria,3,783.0
3,2023-01-01,08001,Industria,4,768.0
4,2023-01-01,08001,Industria,5,2897.0


Al final decido realizar el mismo procedimiento del tramo horario, modificando la columna para identificar a los barrios que corresponden a cada código postal pero será a través de otra tabla. La columna estará relacionada a un `id` para conocer los detalles sobre el código postal completo y al barrio al que pertenece.

In [13]:
mapeo_codigos= {
    "08001": "1",
    "08002": "2",
    "08003": "3",
    "08004": "4",
    "08005": "5",
    "08006": "6",
    "08007": "7",
    "08008": "8",
    "08009": "9",
    "08010": "10",
    "08011": "11",
    "08012": "12",
    "08013": "13",
    "08014": "14",
    "08015": "15",
    "08016": "16",
    "08017": "17",
    "08018": "18",
    "08019": "19",
    "08020": "20",
    "08021": "21",
    "08022": "22",
    "08023": "23",
    "08024": "24",
    "08025": "25",
    "08026": "26",
    "08027": "27",
    "08028": "28",
    "08029": "29",
    "08030": "30",
    "08031": "31",
    "08032": "32",
    "08033": "33",
    "08034": "34",
    "08035": "35",
    "08036": "36",
    "08037": "37",
    "08038": "38",
    "08039": "39",
    "08040": "40",
    "08041": "41",
    "08042": "42"
}

df_consum_2023['codigo_postal']= df_consum_2023['codigo_postal'].map(mapeo_codigos)

df_consum_2023.head()

,fecha,codigo_postal,sector_economico,tramo_horario,consumo
0,2023-01-01,1,Industria,1,644.0
1,2023-01-01,1,Industria,2,702.0
2,2023-01-01,1,Industria,3,783.0
3,2023-01-01,1,Industria,4,768.0
4,2023-01-01,1,Industria,5,2897.0


Realizo la misma tarea con el sector, codificando la columna para cada uno de ellos con lo que será el `id` de cada uno con la tabla correspondiente.

In [14]:
mapeo_sector = {'Industria' : 1,
                'Residencial' : 2,
                'Servicios' : 3,
                'No especificado': 4
                }

df_consum_2023['sector_economico']= df_consum_2023['sector_economico'].map(mapeo_sector)
df_consum_2023.head()

,fecha,codigo_postal,sector_economico,tramo_horario,consumo
0,2023-01-01,1,1,1,644.0
1,2023-01-01,1,1,2,702.0
2,2023-01-01,1,1,3,783.0
3,2023-01-01,1,1,4,768.0
4,2023-01-01,1,1,5,2897.0


Guardo el dataframe en formato csv

In [15]:
df_consum_2023.to_csv('consumo_2023.csv', index= False, encoding='utf-8')

RESUMEN DE LAS TAREAS REALIZADAS:

- Cambio el formato de `Data` a `datetime`
- Se ha eliminado la columna `any`
- Se ha cambiado el tipo de la columna `valor` a `float`
- Se han utilizado los registros `no consta` para tener la suma de los consumos de cada día/sector y para no eliminar los registros.
- Se realizaron modificaciones en los títulos y el idioma de estos para no tener problemas de codificación futuros.
- Se mapearon valores que permitirán trabajar con tablas relacionales posteriormente. Se busca además que los archivos finales tengan menor dimensión (de 22MB a 6MB aproximadamente) Esto se hizo con las columnas `tramo horario`, `codigo_postal` y `sector económico`.
- Se exportó el dataframe a un archivo csv